In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
data = pd.read_csv('Broker_Report_Var10.csv') #загрузим данные

In [3]:
start_date = str(data.date.min())
end_date =str(data.date.max())
print(start_date)
print(end_date) #определим начальную и конечную даты для выгрузки цен

2019-06-19
2020-06-18


In [4]:
from datetime import datetime
from datetime import timedelta

In [5]:
#подгрузим курсы валют
currency = pd.read_csv('USDRUB_190619_200618.txt')
currency_= pd.DataFrame(currency[['<CLOSE>','<DATE>']]) #выделили отдельный датафрейм с тем, что нас интересует
currency_ = currency_.rename(columns={'<CLOSE>':'curr_close', '<DATE>':'date'})

чтобы все цены были в рублях, добавляем столбец с ценой курса на момент закрытия каждого дня \
колебаниями курса внутри дня пренебрегли\
данные скачаны отсюда: https://www.finam.ru/profile/forex/usd-rub/export/ (тот случай, когда вручную быстрее)

In [6]:

currency_['date'] = currency_['date'].apply(lambda x: datetime.strptime(str(x),"%Y%m%d").date())

data['date'] =pd.to_datetime(data['date'])
data = data.set_index('date').join(currency_.set_index('date')).reset_index()

In [7]:
#возьмем разные тикеры и оценим разброс дат

print(data[data['ticker']=='CCL']['date'].nunique())
print(data[data['ticker']=='IRAO']['date'].nunique(),'\n')

print(data.date.nunique())
#даты примерно одинаковые для всех. поэтому сразу возьмем максимум и минимум и по ним будем делать выгрузку

231
233 

261


In [8]:
usd_ticker =list(data.ticker[data.currency=='USD'].value_counts().index)
#для yahoo finance
ru_ticker = list(data.ticker[data.currency=='RUB'].value_counts().index)
#для finam

по каждой из бумаг в нашем датасете нам нужны цены закрытия для подсчета ликвидационной стоимости
воспользуемся скриптами семинара для получения этих данных

In [9]:
#ищем данные для русских тикеров:
# Импорт необходтиых библиотек для анализа данных
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.parse import urlencode
from urllib.request import urlopen

from time import sleep
from random import randint

In [10]:
# Импортируем часовые данные за период 1 год с сегодняшней даты по выбранному активу

zh = pd.DataFrame() #в эту таблицу будем подгружать данные по каждому рублевому тикеру

for i in ru_ticker:
    ticker=i  # Указываем тикер ценной бумаги
    period=8 # указываем необходимый период. в нашем случае 'daily': 8,

    start = str(start_date)
    end = str(end_date)
    
    start_date = datetime.strptime(start, "%Y-%m-%d").date()
    end_date = datetime.strptime(end, "%Y-%m-%d").date()

    #цифровой код акции
    tickers={'ABRD':82460,'AESL':181867,'AFKS':19715,'AFLT':29,'AGRO':399716,'AKRN':17564,'ALBK':82616,'ALNU':81882,'ALRS':81820,'AMEZ':20702,'APTK':13855,'AQUA':35238,'ARMD':19676,'ARSA':19915,'ASSB':16452,'AVAN':82843,'AVAZ':39,'AVAZP':40,'BANE':81757,'BANEP':81758,'BGDE':175840,'BISV':35242,'BISVP':35243,'BLNG':21078,'BRZL':81901,'BSPB':20066,'CBOM':420694,'CHEP':20999,'CHGZ':81933,'CHKZ':21000,'CHMF':16136,'CHMK':21001,'CHZN':19960,'CLSB':16712,'CLSBP':16713,'CNTL':21002,'CNTLP':81575,'DASB':16825,'DGBZ':17919,'DIOD':35363,'DIXY':18564,'DVEC':19724,'DZRD':74744,'DZRDP':74745,'ELTZ':81934,'ENRU':16440,'EPLN':451471,'ERCO':81935,'FEES':20509,'FESH':20708,'FORTP':82164,'GAZA':81997,'GAZAP':81998,'GAZC':81398,'GAZP':16842,'GAZS':81399,'GAZT':82115,'GCHE':20125,'GMKN':795,'GRAZ':16610,'GRNT':449114,'GTLC':152876,'GTPR':175842,'GTSS':436120,'HALS':17698,'HIMC':81939,'HIMCP':81940,'HYDR':20266,'IDJT':388276,'IDVP':409486,'IGST':81885,'IGST03':81886,'IGSTP':81887,'IRAO':20516,'IRGZ':9,'IRKT':15547,'ISKJ':17137,'JNOS':15722,'JNOSP':15723,'KAZT':81941,'KAZTP':81942,'KBSB':19916,'KBTK':35285,'KCHE':20030,'KCHEP':20498,'KGKC':83261,'KGKCP':152350,'KLSB':16329,'KMAZ':15544,'KMEZ':22525,'KMTZ':81903,'KOGK':20710,'KRKN':81891,'KRKNP':81892,'KRKO':81905,'KRKOP':81906,'KROT':510,'KROTP':511,'KRSB':20912,'KRSBP':20913,'KRSG':15518,'KSGR':75094,'KTSB':16284,'KTSBP':16285,'KUBE':522,'KUNF':81943,'KUZB':83165,'KZMS':17359,'KZOS':81856,'KZOSP':81857,'LIFE':74584,'LKOH':8,'LNTA':385792,'LNZL':21004,'LNZLP':22094,'LPSB':16276,'LSNG':31,'LSNGP':542,'LSRG':19736,'LVHK':152517,'MAGE':74562,'MAGEP':74563,'MAGN':16782,'MERF':20947,'MFGS':30,'MFGSP':51,'MFON':152516,'MGNT':17086,'MGNZ':20892,'MGTS':12984,'MGTSP':12983,'MGVM':81829,'MISB':16330,'MISBP':16331,'MNFD':80390,'MOBB':82890,'MOEX':152798,'MORI':81944,'MOTZ':21116,'MRKC':20235,'MRKK':20412,'MRKP':20107,'MRKS':20346,'MRKU':20402,'MRKV':20286,'MRKY':20681,'MRKZ':20309,'MRSB':16359,'MSNG':6,'MSRS':16917,'MSST':152676,'MSTT':74549,'MTLR':21018,'MTLRP':80745,'MTSS':15523,'MUGS':81945,'MUGSP':81946,'MVID':19737,'NAUK':81992,'NFAZ':81287,'NKHP':450432,'NKNC':20100,'NKNCP':20101,'NKSH':81947,'NLMK':17046,'NMTP':19629,'NNSB':16615,'NNSBP':16616,'NPOF':81858,'NSVZ':81929,'NVTK':17370,'ODVA':20737,'OFCB':80728,'OGKB':18684,'OMSH':22891,'OMZZP':15844,'OPIN':20711,'OSMP':21006,'OTCP':407627,'PAZA':81896,'PHOR':81114,'PHST':19717,'PIKK':18654,'PLSM':81241,'PLZL':17123,'PMSB':16908,'PMSBP':16909,'POLY':175924,'PRFN':83121,'PRIM':17850,'PRIN':22806,'PRMB':80818,'PRTK':35247,'PSBR':152320,'QIWI':181610,'RASP':17713,'RBCM':74779,'RDRB':181755,'RGSS':181934,'RKKE':20321,'RLMN':152677,'RLMNP':388313,'RNAV':66644,'RODNP':66693,'ROLO':181316,'ROSB':16866,'ROSN':17273,'ROST':20637,'RSTI':20971,'RSTIP':20972,'RTGZ':152397,'RTKM':7,'RTKMP':15,'RTSB':16783,'RTSBP':16784,'RUAL':414279,'RUALR':74718,'RUGR':66893,'RUSI':81786,'RUSP':20712,'RZSB':16455,'SAGO':445,'SAGOP':70,'SARE':11,'SAREP':24,'SBER':3,'SBERP':23,'SELG':81360,'SELGP':82610,'SELL':21166,'SIBG':436091,'SIBN':2,'SKYC':83122,'SNGS':4,'SNGSP':13,'STSB':20087,'STSBP':20088,'SVAV':16080,'SYNG':19651,'SZPR':22401,'TAER':80593,'TANL':81914,'TANLP':81915,'TASB':16265,'TASBP':16266,'TATN':825,'TATNP':826,'TGKA':18382,'TGKB':17597,'TGKBP':18189,'TGKD':18310,'TGKDP':18391,'TGKN':18176,'TGKO':81899,'TNSE':420644,'TORS':16797,'TORSP':16798,'TRCN':74561,'TRMK':18441,'TRNFP':1012,'TTLK':18371,'TUCH':74746,'TUZA':20716,'UCSS':175781,'UKUZ':20717,'UNAC':22843,'UNKL':82493,'UPRO':18584,'URFD':75124,'URKA':19623,'URKZ':82611,'USBN':81953,'UTAR':15522,'UTII':81040,'UTSY':419504,'UWGN':414560,'VDSB':16352,'VGSB':16456,'VGSBP':16457,'VJGZ':81954,'VJGZP':81955,'VLHZ':17257,'VRAO':20958,'VRAOP':20959,'VRSB':16546,'VRSBP':16547,'VSMO':15965,'VSYD':83251,'VSYDP':83252,'VTBR':19043,'VTGK':19632,'VTRS':82886,'VZRZ':17068,'VZRZP':17067,'WTCM':19095,'WTCMP':19096,'YAKG':81917,'YKEN':81766,'YKENP':81769,'YNDX':388383,'YRSB':16342,'YRSBP':16343,'ZHIV':181674,'ZILL':81918,'ZMZN':556,'ZMZNP':603,'ZVEZ':82001}
    FINAM_URL = "http://export.finam.ru/"
    market = 0  

    params = urlencode([
                        ('market', market), #на каком рынке торгуется бумага
                        ('em', tickers[ticker]), #цифровой символ, который соответствует бумаге.
                        ('code', ticker), #тикер акции
                        ('apply',0), 
                        ('df', start_date.day), #Начальная дата, номер дня (1-31)
                        ('mf', start_date.month - 1), #Начальная дата, номер месяца (0-11)
                        ('yf', start_date.year), #Начальная дата, год
                        ('from', start_date), #Начальная дата полностью
                        ('dt', end_date.day), #Конечная дата, номер дня	
                        ('mt', end_date.month - 1), #Конечная дата, номер месяца
                        ('yt', end_date.year), #Конечная дата, год
                        ('to', end_date), #Конечная дата
                        ('p', period), #Таймфрейм
                        ('f', ticker), #Имя файла
                        ('e', ".csv"), #Расширение сформированного файла
                        ('cn', ticker), #тикер акции	
                        ('dtf', 1), #В каком формате брать даты. Выбор из 5 возможных. См. страницу https://www.finam.ru/profile/moex-akcii/sberbank/export/
                        ('tmf', 1), #В каком формате брать время. Выбор из 4 возможных.
                        ('MSOR', 0), #Время свечи (0 - open; 1 - close)	
                        ('mstime', "on"), #Московское время	
                        ('mstimever', 1), #Коррекция часового пояса	
                        ('sep', 1), #Разделитель полей	(1 - запятая, 2 - точка, 3 - точка с запятой, 4 - табуляция, 5 - пробел)
                        ('sep2', 1), #Разделитель разрядов
                        ('datf', 1), #Формат записи в файл. Выбор из 6 возможных.
                        ('at', 1)]) #Нужны ли заголовки столбцов
   
    url = FINAM_URL + ticker+"_" + str(start_date) + "_" + str(end_date) + ".csv?" + params 
    print("Ссылка на скачивание "+url)
    
    txt=urlopen(url).readlines() 
    local_file = open('{}.txt'.format(i), "w") #задаём файл, в который запишем котировки.
    
    for line in txt:  
        local_file.write(line.strip().decode( "utf-8" )+'\n')	
    local_file.close()
    print("Файл находится в рабочей директории")
    
    sleep(randint(3,8)) #чтобы finam не выдал 403!
    
    #подгрузим данные по каждому рублевому тикеру
    
    df=pd.read_csv("{}.txt".format(i), sep=',') 
    df = df.rename(columns={'<DATE>':'date', '<CLOSE>':'close'})
    df['date'] = df['date'].apply(lambda x: datetime.strptime(str(x),"%Y%m%d").date())
    df['date'] = pd.to_datetime(df['date'])
    df = df[['date','close']]
    df = data[data.ticker=='{}'.format(i)].set_index('date').join(df.set_index('date')).reset_index()
    
    zh = zh.append(df) #сохраняем данные в одной таблице
        
zh.head()

Ссылка на скачивание http://export.finam.ru/IRAO_2019-06-19_2020-06-18.csv?market=0&em=20516&code=IRAO&apply=0&df=19&mf=5&yf=2019&from=2019-06-19&dt=18&mt=5&yt=2020&to=2020-06-18&p=8&f=IRAO&e=.csv&cn=IRAO&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=1&sep2=1&datf=1&at=1
Файл находится в рабочей директории
Ссылка на скачивание http://export.finam.ru/TATN_2019-06-19_2020-06-18.csv?market=0&em=825&code=TATN&apply=0&df=19&mf=5&yf=2019&from=2019-06-19&dt=18&mt=5&yt=2020&to=2020-06-18&p=8&f=TATN&e=.csv&cn=TATN&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=1&sep2=1&datf=1&at=1
Файл находится в рабочей директории
Ссылка на скачивание http://export.finam.ru/GMKN_2019-06-19_2020-06-18.csv?market=0&em=795&code=GMKN&apply=0&df=19&mf=5&yf=2019&from=2019-06-19&dt=18&mt=5&yt=2020&to=2020-06-18&p=8&f=GMKN&e=.csv&cn=GMKN&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=1&sep2=1&datf=1&at=1
Файл находится в рабочей директории
Ссылка на скачивание http://export.finam.ru/HYDR_2019-06-19_2020-06-18.csv?market=0

,date,time,action,ticker,price,currency,volume,curr_close,close
0,2019-06-20,12:13:28,S,IRAO,4.96,RUB,4,62.8381,4.9575
1,2019-06-21,14:31:42,B,IRAO,4.92,RUB,3,63.0000,4.9250
2,2019-06-21,17:24:12,B,IRAO,4.92,RUB,9,63.0000,4.9250
3,2019-06-24,14:50:30,S,IRAO,5.00,RUB,10,62.5410,5.0515
4,2019-06-24,15:11:10,B,IRAO,5.09,RUB,10,62.5410,5.0515


In [11]:
%matplotlib inline
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import datetime as dt

In [12]:
usd_ticker

['CCL', 'MSFT', 'JNJ']

In [13]:
ol = pd.DataFrame()
for i in usd_ticker:
    dff  = pd.DataFrame(yf.download(usd_ticker[0], start=start_date, end=end_date).Close).reset_index().\
    rename(columns={'Date':'date', 'Close':'close'})
    dff = data[data.ticker=='{}'.format(i)].set_index('date').join(dff.set_index('date')).reset_index()
    ol = ol.append(dff)
ol.head()
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,date,time,action,ticker,price,currency,volume,curr_close,close
0,2019-06-19,13:08:23,B,CCL,52.82,USD,1,63.6930,52.840000
1,2019-06-19,14:27:25,B,CCL,52.85,USD,7,63.6930,52.840000
2,2019-06-19,14:41:41,B,CCL,52.88,USD,9,63.6930,52.840000
3,2019-06-19,16:30:30,B,CCL,52.87,USD,8,63.6930,52.840000
4,2019-06-20,13:25:59,S,CCL,46.98,USD,1,62.8381,48.799999


In [14]:
ol['close'] = ol['close'].fillna(method='pad')
zh['close'] = zh['close'].fillna(method='pad') #на случай, если в ценах выгрузки были пропуски, заменим их на предыдущее значение

In [15]:
ol.price = round(ol.price*ol.curr_close,2)
ol.close = round(ol.close*ol.curr_close,2) #приведем цены покупки и цены закрытия к рублям

In [16]:
ol['close'] = round(ol['close'],2)
zh['close'] = round(zh['close'],2) #округлим, чтобы удобнее работать

In [17]:
data_ = zh.append(ol) #объединим с рублевыми тикерами

In [18]:
data_ = data_.drop(['currency'],axis=1) #избавимся от валюты, т.к. к рублям привели выше
data_ = data_.reset_index()
data_ = data_.drop(['index'],axis=1)

In [19]:
tickers = list(data_.ticker.unique())
print(tickers)
dates = list(data_.date.unique())
# print(dates)

['IRAO', 'TATN', 'GMKN', 'HYDR', 'MTSS', 'CCL', 'MSFT', 'JNJ']


In [20]:
data_['curr_close'] = round(data_['curr_close'],2)

In [21]:
dates  = list(data_.date.unique())

In [22]:
#отсортируем и сразу посчитаем в накопленную прибыль те акции, что были первой же датой проданы.

In [23]:
o = pd.DataFrame()
c = pd.DataFrame()
#создадим две таблицы, куда будем прикреплять открытые и закрытые позиции соответственно

In [24]:
len(tickers)

8

Решили пойти методом последовательного итерирования по датафрейму. перебираются данные для каждого тикера по всем датам  (по очереди).  получается 8*5к итераций, перебор занимает около 20 минут.

In [26]:
# оставим один актив и несколько дат
for ticker in tickers:

    df_ = data_[data_.ticker == '{}'.format(ticker)]
    df_ = df_.reset_index(drop = True)
    
    for d in range(len(dates)): 
        df = df_ #перед тем, как выбрать дату, обращаемся ко всему датасету с выбранным тикером

        df = df[df.date==dates[d]] #выбираем дату

        if df.shape[0]>0: #проверяем, работали ли мы с тикером в эту дату 
            print('дата: ', dates[d], 'тикер: ',ticker)
            print(df.shape)

            # ищем остаток по дню    
            surplus = (np.select([df.action == "B", df.action == "S"], [1, -1]) * df.volume).sum()
            print('излишек: ',surplus)
            


            if surplus<0:
            # значит, непарные сделки находятся в продажах
                df_w = df[df.action == "S"]
            elif surplus>0:
                # значит, непарные сделки находятся в покупках
                df_w = df[df.action == "B"]
            else:
                df_w = df
                #выходит, у нас нет непарных сделок, можем сразу считать накопленную сумму

        # рассчитываем от конца нашей небольшой таблицы накопленный объем, чтобы найти разделитель границы
            cum_vol_rev = df_w.iloc[::-1].volume.cumsum()


            df_w['cum_vol_rev'] = cum_vol_rev
            border_row = 0
            for i in reversed(range(df_w.shape[0])):
                
                if df_w.cum_vol_rev.iat[i] >= abs(surplus):
                    border_row = i # запоминаем номер строки
                    break
                    
            #формируем две таблицы, пограниченое значение определяем в обе
                    
            df_w_closed = df_w.iloc[:(border_row + 1),:]
            # некрытые (то есть непарные) позиции на продажу
            df_w_opened = df_w.iloc[border_row:,:]
            
            vol_mov2closed = df_w_opened.cum_vol_rev.iat[0] - abs(surplus) # считаем избыток над остатком

            df_w_closed.volume.iat[-1] = vol_mov2closed # переносим в closed избыток
            df_w_opened.volume.iat[0] = df_w_opened.volume.iat[0] - vol_mov2closed # уменьшаем opened на остаток


            # уберем временный столбец cum_vol_rev
            df_w_closed = df_w_closed.drop(['cum_vol_rev'], axis=1)
            df_w_opened = df_w_opened.drop(['cum_vol_rev'], axis=1)

            # объединяем таблицы (если остаток был равен нулю, )
            if surplus<0:
                df_w_paired = pd.concat([df[df.action == "B"], df_w_closed])
            if surplus>0:
                df_w_paired = pd.concat([df[df.action == "S"], df_w_closed])
            if surplus==0:
                df_w_paired = df_w


            # этот датафрейм можно использовать для рассчета накопленной прибыли
            df_w_paired.sort_index()   

            #посчитаем накопленную прибыль
            df_w_paired['cumulative_profit'] = sum(df_w_paired.price.loc[df_w_paired.action == "S"] *\
                                                   df_w_paired.volume.loc[df_w_paired.action == "S"]) - \
            sum(df_w_paired.price.loc[df_w_paired.action == "B"]*\
                df_w_paired.volume.loc[df_w_paired.action == "B"])

            c = c.append(df_w_paired)#добавляем к таблице для расчета накопленной прибыли

            # посчитаем ликвидационную стоимость 
            liq_cost = df_w_opened['close']*df_w_opened['volume']
            df_w_opened['liq_cost'] = liq_cost #в зависимости от типа действия изменим знак в итоговой таблице
            
            o=o.append(df_w_opened) #добавляем к таблице для ра счета ликвидационной стоимости       

дата:  2019-06-20T00:00:00.000000000 тикер:  IRAO
(1, 8)
излишек:  -4
дата:  2019-06-21T00:00:00.000000000 тикер:  IRAO
(2, 8)
излишек:  12
дата:  2019-06-24T00:00:00.000000000 тикер:  IRAO
(3, 8)
излишек:  -8
дата: 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

 2019-06-25T00:00:00.000000000 тикер:  IRAO
(2, 8)
излишек:  -13
дата:  2019-06-26T00:00:00.000000000 тикер:  IRAO
(2, 8)
излишек:  -1
дата:  2019-06-27T00:00:00.000000000 тикер:  IRAO
(3, 8)
излишек:  10


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

дата:  2019-06-28T00:00:00.000000000 тикер:  IRAO
(4, 8)
излишек:  15
дата:  2019-07-01T00:00:00.000000000 тикер:  IRAO
(6, 8)
излишек:  1


...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


 2019-12-20T00:00:00.000000000 тикер:  JNJ
(2, 8)
излишек:  -8
дата:  2019-06-19T00:00:00.000000000 тикер:  JNJ
(5, 8)
излишек:  -26
дата:  2019-11-04T00:00:00.000000000 тикер:  JNJ
(4, 8)
излишек:  -10


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


дата:  2019-12-31T00:00:00.000000000 тикер:  JNJ
(1, 8)
излишек:  -5
дата:  2020-01-02T00:00:00.000000000 тикер:  JNJ
(3, 8)
излишек:  -15
дата:  2020-01-07T00:00:00.000000000 тикер:  JNJ
(2, 8)
излишек:  -11
дата:  2020-03-09T00:00:00.000000000 тикер:  JNJ
(1, 8)
излишек:  8
дата:  2020-05-01T00:00:00.000000000 тикер:  JNJ
(1, 8)
излишек:  1
дата:  2020-05-11T00:00:00.000000000 тикер:  JNJ
(7, 8)
излишек:  12
дата:  2020-02-24T00:00:00.000000000 тикер:  JNJ
(1, 8)
излишек:  -4
дата:  2020-06-12T00:00:00.000000000 тикер:  JNJ
(3, 8)
излишек:  13


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


**Итоговая обработка данных**

In [27]:
print(o.date.nunique())
print(data_.date.nunique())
print(c.date.nunique())
#проверим совпадение данных

261
261
261


In [28]:
o['v'] = np.select([o.action == "B", o.action == "S"], [1, -1]) * o.liq_cost

In [29]:
o.head()

,date,time,action,ticker,price,volume,curr_close,close,liq_cost,v
0,2019-06-20,12:13:28,S,IRAO,4.96,4,62.84,4.96,19.84,-19.84
1,2019-06-21,14:31:42,B,IRAO,4.92,3,63.00,4.92,14.76,14.76
2,2019-06-21,17:24:12,B,IRAO,4.92,9,63.00,4.92,44.28,44.28
5,2019-06-24,16:30:25,S,IRAO,4.93,8,62.54,5.05,40.40,-40.40
6,2019-06-25,10:05:41,S,IRAO,4.66,8,62.84,4.82,38.56,-38.56


In [30]:
liq_cost_row = pd.DataFrame(o.groupby('date', as_index=False).liq_cost.sum())

In [31]:
liq_cost_row.head(3)

,date,liq_cost
0,2019-06-19,205297.94
1,2019-06-20,113049.67
2,2019-06-21,283900.53


In [32]:
c.head(3)

,action,close,cum_vol_rev,cumulative_profit,curr_close,date,price,ticker,time,volume
0,S,4.96,NaN,0.0,62.84,2019-06-20,4.96,IRAO,12:13:28,0
1,B,4.92,NaN,0.0,63.00,2019-06-21,4.92,IRAO,14:31:42,0
4,B,5.05,NaN,-0.9,62.54,2019-06-24,5.09,IRAO,15:11:10,10


In [33]:
c.cumulative_profit.sum()

117948.74999999914

In [34]:
profit = pd.DataFrame(c.groupby('date', as_index=False).cumulative_profit.sum())
profit['cum'] = profit.cumulative_profit.cumsum() #ведем накопленную прибыль по датам в отдельный столбец
out = profit.set_index('date').join(liq_cost_row.set_index('date')).reset_index()
#сформируем итоговую таблицу

In [35]:
out.head()

,date,cumulative_profit,cum,liq_cost
0,2019-06-19,0.00,0.00,205297.94
1,2019-06-20,1987.88,1987.88,113049.67
2,2019-06-21,550.70,2538.58,283900.53
3,2019-06-24,-1949.26,589.32,156782.67
4,2019-06-25,-54.78,534.54,59494.72


In [36]:
out = out.drop(['cumulative_profit'], axis=1)
list(out.columns)
cols = ['date', 'liq_cost','cum']
out = out[cols] #выведем в нужном нам порядке
out.head()

,date,liq_cost,cum
0,2019-06-19,205297.94,0.00
1,2019-06-20,113049.67,1987.88
2,2019-06-21,283900.53,2538.58
3,2019-06-24,156782.67,589.32
4,2019-06-25,59494.72,534.54


In [37]:
import csv
out.to_csv('mini_project_var10.csv',index = None, header = True, decimal = '.')

In [38]:
#проверим
check = pd.read_csv('mini_project_var10.csv')
check.head()

,date,liq_cost,cum
0,2019-06-19,205297.94,0.00
1,2019-06-20,113049.67,1987.88
2,2019-06-21,283900.53,2538.58
3,2019-06-24,156782.67,589.32
4,2019-06-25,59494.72,534.54
